In [15]:
%load_ext autoreload
%autoreload 2


import keras
from keras.models import load_model
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.utils.eval import evaluate
from keras_retinanet.utils.gpu import setup_gpu
from keras_retinanet.models.mobilenet import custom_objects
setup_gpu('0')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 Physical GPUs, 1 Logical GPUs


In [16]:
classes = "D:\Amirhosein\Object_Detection\\tag-detection-RetinaNet\\tag-detection-retinanet_MobileNet\dataset\\classes.csv"
val_path = "D:\Amirhosein\Object_Detection\\tag-detection-RetinaNet\\tag-detection-retinanet_MobileNet\dataset\\val.csv"
test_path = "D:\Amirhosein\Object_Detection\\tag-detection-RetinaNet\\tag-detection-retinanet_MobileNet\dataset\\test.csv"
model_path = "D:\Amirhosein\Object_Detection\\tag-detection-RetinaNet\\tag-detection-retinanet_MobileNet\\snapshots\mobilenet_a1_s8_rdc2\mobilenet_a1_s8_rdc2_best.h5"
# model_path = "D:\Amirhosein\Object_Detection\keras-retinanet\snapshots\retinanet_resnet_09_2.h5"

test_image_data_generator = keras.preprocessing.image.ImageDataGenerator()

    # create a generator for testing data
test_generator = CSVGenerator(
    csv_data_file=test_path,
    csv_class_file=classes,
    image_data_generator=test_image_data_generator,
    batch_size=2,
    )

model = load_model(model_path, custom_objects=custom_objects)


tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

c:\Users\a.pourdavoud\Anaconda3\envs\example\lib\site-packages\keras\engine\training_utils.py:819: UserWarning: Output non_maximum_suppression_2 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to non_maximum_suppression_2.
  'be expecting any data to be passed to {0}.'.format(name))


In [19]:
import tensorflow as tf
from tensorflow.keras import optimizers
tms = tf.saved_model.save(model, "model")

ValueError: Expected a Trackable object for export, got <keras.engine.training.Model object at 0x00000249E7B0A548>.

In [24]:
# run evaluation
logs = evaluate(
    generator=test_generator,
    model=model,
    iou_threshold=0.5,
    score_threshold=0.05,
    max_detections=100,
    save_path='output')

Parsing annotations: : 100%|██████████| 88/88 [00:00<00:00, 682.17it/s]

88 instances of class tag with:
recall_of_tag: 0.8295
precision_of_tag: 0.8391
accuracy: 0.8295
average_precision_of_tag: 0.7425
inference_time: 0.1351
mAP: 0.7425



In [298]:
from keras_retinanet.utils.visualization import draw_box, draw_caption
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
import time

In [ ]:
df = pd.read_csv(test_path, header=None )
df.head()

In [ ]:
img_test = df.iloc[:][0]
img_test = img_test.values

In [295]:
def tag_search_retinanet(img, output_path):
        scale = 1.0
        bb_list = []
        label_list= [0]
        labels_to_names = {0:'tag'}
        image_path = img

        img = cv2.imread(img)
        img_scaled = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img_scaled = cv2.resize(img, None, fx=scale, fy=scale)

        start = time.time()
        _, _, detections = model.predict_on_batch(np.expand_dims(img_scaled, axis=0))
        inference_time = time.time() - start
        
        predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
        scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]
        detections[:, :4] /= scale

        for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
            if score < 0.9:
                continue
            if label not in label_list:
                continue
            b = detections[0, idx, :4].astype(int)
            bb_list.append([(b[0], b[1], b[2], b[3]), score])
        
        # selected_indices = tf.image.non_max_suppression([i for i, _ in bb_list], [i for _, i in bb_list], max_output_size=2, iou_threshold=0.5)
        # selected_boxes = tf.gather([i for i, _ in bb_list], selected_indices)
        # print(selected_boxes.shape)
        # draw_boxes(img, selected_boxes, color=(100, 100, 250))
        for b, score in bb_list:
            draw_box(img, b, color=(100, 100, 250))
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(img, b, caption)

        img_name = image_path.split('\\')[-1]
        output_path = output_path + f'/output/out_{img_name}'
        cv2.imwrite(output_path, img)

In [292]:
from tqdm import tqdm as t
labels_to_names = {0: 'tag'}
output_path = "D:\Amirhosein\Object_Detection\keras-retinanet-mobilenet"
for image_path in t(img_test[3:4]):
    tag_search_retinanet(image_path, output_path)

100%|██████████| 1/1 [00:00<00:00,  5.21it/s]
